In [23]:
import cv2
import numpy as np
import osmapi

from geopy.distance import geodesic

In [24]:
COLORSCHEME = {
        'building' : (0, 0, 255),
        'waterway' : (255, 0, 0),
        'natural' : (0, 255, 0),
        'leisure' : (0, 128, 128),
        'highway' : (0, 128, 128)
    }

LIMIT_WIDTH = LIMIT_HEIGHT = 1500

api = osmapi.OsmApi()

In [25]:
def area_sectorization(min_lon : float, min_lat : float, max_lon : float, max_lat : float) -> list[list[float]]:
    width = int(geodesic((min_lat, min_lon), (min_lat, max_lon)).kilometers * 1000)
    height = int(geodesic((min_lat, min_lon), (max_lat, min_lon)).kilometers * 1000)
    width_ratio = int(np.ceil(width / LIMIT_WIDTH))
    height_ratio = int(np.ceil(height / LIMIT_HEIGHT))
    delta_lon = (max_lon - min_lon) / width_ratio
    delta_lat = (max_lat - min_lat) / height_ratio
    sectors = []
    for i in range(width_ratio):
        for j in range(height_ratio):
            sectors.append([min_lon + i*delta_lon, min_lat + j*delta_lat,
                            min_lon + (i+1)*delta_lon, min_lat + (j+1)*delta_lat])
    return sectors


In [26]:
sectors = area_sectorization(30.3083, 59.8911, 30.3377, 59.9047)

sectors

[[30.3083, 59.8911, 30.323, 59.8979],
 [30.3083, 59.8979, 30.323, 59.9047],
 [30.323, 59.8911, 30.3377, 59.8979],
 [30.323, 59.8979, 30.3377, 59.9047]]

In [27]:
def get_map_visualization(min_lon : float, min_lat : float, max_lon : float, max_lat : float) -> np.array:
    height = int(geodesic((min_lat, min_lon), (min_lat, max_lon)).kilometers * 1000)
    width = int(geodesic((min_lat, min_lon), (max_lat, min_lon)).kilometers * 1000)
    image = np.zeros((width, height, 3), np.uint8)
    sectors = area_sectorization(min_lon, min_lat, max_lon, max_lat)
    all_nodes = []
    for sector in sectors:
        all_nodes += api.Map(*sector)
    print(len(all_nodes))
    nodes = {node['data']['id'] : node['data'] for node in all_nodes if node['type'] == 'node'}
    ways = {node['data']['id'] : node['data'] for node in all_nodes if node['type'] == 'way'}
    for way_id in ways.keys():
        way = ways[way_id]
        way_tags = way['tag']
        visible = False
        for tag in way_tags:
            if tag in COLORSCHEME.keys():
                visible = True
                main_tag = tag
                break

        if not visible: continue

        node_cords = []
        for node_id in way['nd']:
            node = nodes[node_id]
            node_cord_y = int(geodesic((max_lat, min_lon), (node['lat'], min_lon)).kilometers * 1000)
            node_cord_x = int(geodesic((max_lat, min_lon), (max_lat, node['lon'])).kilometers * 1000)
            node_cords.append([node_cord_x, node_cord_y])
        node_cords = np.array(node_cords, np.int32)
        node_cords = node_cords.reshape((-1, 1, 2))
        cv2.polylines(image, [node_cords], True, COLORSCHEME[main_tag])
    return image


In [28]:
result = get_map_visualization(30.2803, 59.9266, 30.3391, 59.9538)
cv2.imwrite('map.png', result)

135764


True